In [2]:
from pathlib import Path
from typing import Union
import itk
import os
import time
from scripts.slices import *
from scripts.visualization import visualize_array

In [3]:
BASE_PATH = Path(os.getcwd())
BASE_PATH

PosixPath('/home/estudiante/curso-imageproc/medical-image-processing-project')

In [123]:
names = ["10", "11", "12", "14", "16", "17", "18", "28"]
name = names[4]
extension = "nii.gz"
input_image = BASE_PATH / f"train_nifti/{name}/p{name}_PET.{extension}"

In [124]:
dimensions = 3
pixel_type_in = itk.F
pixel_type_out = itk.UC
image_type_in = itk.Image[pixel_type_in, dimensions]
image_type_out = itk.Image[pixel_type_out, dimensions]

# Step 1: Read the image as float
reader = itk.ImageFileReader[image_type_in].New()
reader.SetFileName(str(input_image))
reader.Update()
image = reader.GetOutput()

# Step 2: Rescale to unsigned short
rescaler = itk.RescaleIntensityImageFilter[image_type_in, image_type_out].New()
rescaler.SetInput(image)
rescaler.SetOutputMinimum(0)
rescaler.SetOutputMaximum(255)
rescaler.Update()

image = reader.GetOutput()
size = itk.size(image)
np_image = itk.GetArrayFromImage(image)
np_image.shape

(241, 144, 144)

In [125]:
np_image.max()

np.float32(51944.445)

In [126]:
from scripts.slices import extract_initial_slice, extract_final_slice

initial_slice = extract_initial_slice(np_image, intensity_threshold=50, min_sequence_lenght=10)
initial_slice

220

In [ ]:
leg_slice, other = extract_final_slice(np_image, initial_slice, intensity_threshold=3, leg_intensity_threshold=50)
print(leg_slice, other)

In [ ]:
visualize_array(np_image[initial_slice,:,:])

In [ ]:
np_image_2 = np_image[0:initial_slice+1,:,:]

In [ ]:
np_image_2.shape

In [ ]:
from scripts.slices import kmeans_labels
num_pixels_arr = []
for z in range(np_image_2.shape[0]):
    slice_array = np_image_2[z,:,:]
    num_pixels_arr.append((slice_array > 100).sum())
    
num_pixels_arr = np.array(num_pixels_arr).reshape(-1,1)
labels = kmeans_labels(num_pixels_arr, n_clusters=3)
for i,val in enumerate(labels):
    print(i, " : ", val)
    

In [ ]:
visualize_array(np_image[141,:,:])

In [10]:
from scripts.image_processing import *

names = ["10", "11", "12", "14", "16", "17", "18", "28"]
name = names[5]
extension = "nii.gz"
input_image_pet = BASE_PATH / f"train_nifti/{name}/p{name}_PET.{extension}"
input_image_ct = BASE_PATH / f"train_nifti/{name}/p{name}_CT.{extension}"

In [11]:
get_slices_range(input_image_ct, input_image_pet)

(182, 356, 133)

In [1]:
from scripts.image_processing import *